In [0]:
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras import models
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import PIL.Image

'''
データの設定
'''
# 全ての画像を1/255でスケーリング
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 20

train_generator = train_datagen.flow_from_directory(
    directory= "train",      # ターゲットディレクトリ
    target_size= (150, 150),                                            # 全ての画像のサイズを(150, 150)に変更
    batch_size= batch_size, # バッチサイズ
    class_mode= "binary"    # binary_crossentropyを使用するため二値のラベルが必要
)
validation_generator = test_datagen.flow_from_directory(
    directory= "validation",
    target_size= (150, 150),
    batch_size= batch_size,
    class_mode= "binary"
)

'''
モデルの構築
'''
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
# model.summary()
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])

'''
学習
'''
early_stopping = EarlyStopping(monitor= "val_loss", patience= 10, verbose= 1)

history = model.fit_generator(
    train_generator,
    steps_per_epoch= 2000 / batch_size,                   # 2000のサンプルをバッチサイズ20で学習するため、100回のループ
    epochs= 30,
    validation_data= validation_generator,
    validation_steps= 50,
    callbacks= [early_stopping]
)
model.save("cats_and_dogs_small_1.h5")



Using TensorFlow backend.


In [1]:
'''
評価
'''
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, label="Training acc", color= "pink")
plt.plot(epochs, val_acc, label="Validation acc", color= "red")
plt.title("Training and Validation accuracy")
plt.legend()

plt.figure()

plt.plot(epochs, loss, label="Training loss", color= "gray")
plt.plot(epochs, val_loss, label="Validation loss", color= "blue")
plt.title("Training and Validation loss")
plt.legend()

plt.show()

NameError: ignored